In [2]:
import os
import mailbox
import sys
import pprint
import pandas as pd

In [2]:
def extract_message_id(mbox_file):
    mbox = mailbox.mbox(mbox_file)
    for key in mbox.iterkeys():
        try:
            message = mbox[key]
        except mbox.errors.MessageParseError:
            continue  # The message is malformed. Just leave it.
        message_id = str(message['Message-ID'])
        data_row = ['non-phishing'] + message_id.lstrip('<').rstrip('>').split('@')
        return data_row

data_all_rows = []
for root, dirs, files in os.walk("./spamassasin/"):
    for file in files:
        if file.endswith(".mbox"):
            data_all_rows.append(extract_message_id(os.path.join(root, file)))
data_all_rows = list(filter(None, data_all_rows))

In [4]:
df = pd.DataFrame(data_all_rows, columns = ['class_label','LHS','RHS'])

In [5]:
df.to_csv(r'./non_phishing.csv', index = False)

In [57]:
df = pd.read_csv('phishingid.csv',header=None)

#Create the data list
data = []
for index, row in df.iterrows():
    data.append(['phishing']+row[0][2:-1].split('@'))
    
#Remove inaccurate values
for x in data:
    if len(x) != 3:
        data.remove(x)
    
#Create the dataframe and export to csv
pd.DataFrame(data,columns = ['class_label','LHS','RHS']).to_csv('phishing.csv',index=False)